# Module #4 -- Assignment

# Instructions
Using all the knowledge acquired mainly in this module to re-experiment what we discussed in this module using the raw IULA data



## Preparation

- Download the IULA dataset from [here](https://repositori.upf.edu/handle/10230/20052) (the link to the file is in the View document tab next to the Description)
- The raw data is in EN/*/*-plain.txt in all the subfolders.

## Your task is to

### 1. Concatenate all the plain.txt into one file, then
  - convert all the punctuations and non-English characters into space;
  - collapse any trailing spaces into one single space;
  - transform all the characters into lowercase;
  - do not stem any of them.
  
### 2. Using the product of (1.) and recreate the table shown on page #22.
  - If the value is not the same, give hypotheses what is the root cause that makes the page #22 in the slide incorrect
  - Complete the 10^9P(x|w)P(w) using Norvig's prob and your IULA (page #30)
  
### 3. Use your calculated IULA and complete the calculation like that on page #37

---

### The Essentail Library

In [1]:
import pandas as pd # do some data
import numpy as np
import multiprocessing as mp
import functools
import itertools
from string import ascii_lowercase
import os
from pathlib import Path
import re

---

# Assignment Part

## 1. Concatenate all the plain.txt into one file, then

In [2]:
path = "E:\CMU\953\IR481\Module4\IULA\EN"
os.chdir(path)
root_folder = os.listdir()

context = ''
for file in root_folder:
    folder = f"E:\CMU\953\IR481\Module4\IULA\EN\{file}"
    os.chdir(folder)
    plan_text = os.listdir()[1]
    file_path = f"{folder}\{plan_text}"
    temp = Path(file_path).read_text('utf-8')
    temp = temp.replace('\n', '')
    context += temp

print(context[1:300])

trategies on the future of the chlorine industry C. Hendriks and D. Papameletiou Issue: Production and use of chlorine are of key importance for the chemical industry, with a large number of chlorinated compounds on the market, and nearly 60% of the global sales of chemical products being directly 


### convert all the punctuations and non-English characters into space;

In [3]:
context = re.sub('[^A-Za-z]'," ",context)
print(context[1:300])

trategies on the future of the chlorine industry C  Hendriks and D  Papameletiou Issue  Production and use of chlorine are of key importance for the chemical industry  with a large number of chlorinated compounds on the market  and nearly     of the global sales of chemical products being directly 


### collapse any trailing spaces into one single space;

In [4]:
context = " ".join(context.split())
print(context[1:300])

trategies on the future of the chlorine industry C Hendriks and D Papameletiou Issue Production and use of chlorine are of key importance for the chemical industry with a large number of chlorinated compounds on the market and nearly of the global sales of chemical products being directly or indire


### transform all the characters into lowercase (do not stem any of them.)

In [5]:
context = context.lower()
print(context[1:300])

trategies on the future of the chlorine industry c hendriks and d papameletiou issue production and use of chlorine are of key importance for the chemical industry with a large number of chlorinated compounds on the market and nearly of the global sales of chemical products being directly or indire


---

## 2. Using the product of (1.) and recreate the table shown on page #22.

### recreate the table show on page#22

In [6]:
IULA = pd.DataFrame(
    [
        ['defeat',context.count('defeat')],
        ['decet',context.count('decet')],
        ['defect',context.count('defect')],
        ['deft',context.count('deft')],
        ['defer',context.count('defer')],
        ['Deeft',context.count('Deeft')]
    ],
    columns=['word','frequency']
)
IULA_pop = 2.1e6
IULA['P(w)'] = IULA['frequency']/IULA_pop
IULA['rank'] = IULA['frequency'].rank(ascending=False).astype(int)

IULA.head(6)

,word,frequency,P(w),rank
0,defeat,12,0.000006,2
1,decet,0,0.000000,5
2,defect,198,0.000094,1
3,deft,0,0.000000,5
4,defer,11,0.000005,3
5,Deeft,0,0.000000,5


### Complete the 10^9P(x|w)P(w) using Norvig's prob and your IULA

In [7]:
norvig = pd.read_csv(
    'http://norvig.com/ngrams/count_1edit.txt',
    sep='\t',
    encoding = "ISO-8859-1",
    header=None
)
norvig.columns = ['term', 'edit']
norvig = norvig.set_index('term')

In [8]:
norvig_orig = pd.read_csv(
    'http://norvig.com/ngrams/count_big.txt',
    sep='\t',
    encoding = "ISO-8859-1",
    header=None
)
norvig_orig = norvig_orig.dropna()
norvig_orig.columns=['term','freq']

In [9]:
def get_count(c):
    return norvig_orig.apply(lambda x: x.term.count(c) * x.freq, axis=1).sum()

In [10]:
character_set = list(map(''.join, itertools.product(ascii_lowercase, repeat=1))) + list(map(''.join, itertools.product(ascii_lowercase, repeat=2)))

In [11]:
freq_list = []
for i in character_set :
    freq_list.append(get_count(i))

In [12]:
freq_df = pd.DataFrame([character_set, freq_list], index=['char', 'freq']).T
freq_df = freq_df.set_index('char')
IULA['P(x|w)'] = [
    (norvig.loc['e|ea'].values / freq_df.loc['ea'].values)[0],
    (norvig.loc['f|c'].values / freq_df.loc['c'].values)[0],
    (norvig.loc['e|ec'].values / freq_df.loc['ec'].values)[0],
    (norvig.loc['e| '].values / freq_df.loc['e'].values)[0],
    (norvig.loc['t|r'].values / freq_df.loc['r'].values)[0],
    (norvig.loc['fe|ef'].values / freq_df.loc['ef'].values)[0]
]
IULA.head(6)

,word,frequency,P(w),rank,P(x|w)
0,defeat,12,0.000006,2,0.012834
1,decet,0,0.000000,5,0.000028
2,defect,198,0.000094,1,0.003167
3,deft,0,0.000000,5,0.000003
4,defer,11,0.000005,3,0.000036
5,Deeft,0,0.000000,5,0.003311


### Complete the 10^9P(x|w)P(w) using Norvig's prob and your IULA

In [13]:
IULA['10e9 P(x|w)P(w)'] = 1e9 * IULA['P(w)'] * IULA['P(x|w)']
IULA.head(6)

,word,frequency,P(w),rank,P(x|w),10e9 P(x|w)P(w)
0,defeat,12,0.000006,2,0.012834,73.337097
1,decet,0,0.000000,5,0.000028,0.000000
2,defect,198,0.000094,1,0.003167,298.593664
3,deft,0,0.000000,5,0.000003,0.000000
4,defer,11,0.000005,3,0.000036,0.186141
5,Deeft,0,0.000000,5,0.003311,0.000000


---

## 3. Use your calculated IULA and complete the calculation like that on page #38

giving the sentence

1.'a defeat' 

2.'a defect' 

3.'defeat free' 

4.'defect free'

In [20]:
mock_sentence = [
    ['a', context.count('a')], 
    ['free', context.count('free')], 
    ['defeat', context.count('defeat')], 
    ['defect', context.count('defect')],
    ['a defeat', context.count('a defeat')], 
    ['a defect', context.count('a defect')], 
    ['defeat free', context.count('defeat free')], 
    ['defect free', context.count('defect free')]
]
print(mock_sentence)

[['a', 425037], ['free', 373], ['defeat', 12], ['defect', 198], ['a defeat', 0], ['a defect', 21], ['defeat free', 0], ['defect free', 0]]


bigram model language

In [54]:
mock_sentence = [
    ['a', context.count('a'),'',''], 
    ['free', context.count('free'),'',''], 
    ['defeat', context.count('defeat'),'',''], 
    ['defect', context.count('defect'),'',''],
    ['a defeat', context.count('a defeat'),'P(defeat | a)', 'C(a defeat)/C(defeat)'], 
    ['a defect', context.count('a defect'),'P(defect | a)', 'C(a defect)/C(defect)'], 
    ['defeat free', context.count('defeat free'),'P(free | defeat)', 'C(defeat free) / C(free)'], 
    ['defect free', context.count('defect free'),'P(free | defect)','C(defect free) / C(free)']
]

In [55]:
IULA_cal = pd.DataFrame(
    mock_sentence,
    columns=['word','frequency','P(Wk|Wk-1)','C(Wk-1|Wk)/C(Wk-1)']
)

In [56]:
print("candidate word")
IULA_cal.head(4)

candidate word


,word,frequency,P(Wk|Wk-1),C(Wk-1|Wk)/C(Wk-1)
0,a,425037,,
1,free,373,,
2,defeat,12,,
3,defect,198,,


In [57]:
print("focus data")
IULA_cal.tail(4)

focus data


,word,frequency,P(Wk|Wk-1),C(Wk-1|Wk)/C(Wk-1)
4,a defeat,0,P(defeat | a),C(a defeat)/C(defeat)
5,a defect,21,P(defect | a),C(a defect)/C(defect)
6,defeat free,0,P(free | defeat),C(defeat free) / C(free)
7,defect free,0,P(free | defect),C(defect free) / C(free)


In [61]:
print(IULA_cal['frequency'][4]/IULA_cal['frequency'][2])

0.0


In [63]:
mock_sentence = [
    ['a', context.count('a'),'','',''], 
    ['free', context.count('free'),'','',''], 
    ['defeat', context.count('defeat'),'','',''], 
    ['defect', context.count('defect'),'','',''],
    ['a defeat', context.count('a defeat'),'P(defeat | a)', 'C(a defeat)/C(defeat)',IULA_cal['frequency'][4]/IULA_cal['frequency'][2]], 
    ['a defect', context.count('a defect'),'P(defect | a)', 'C(a defect)/C(defect)',IULA_cal['frequency'][5]/IULA_cal['frequency'][3]], 
    ['defeat free', context.count('defeat free'),'P(free | defeat)', 'C(defeat free) / C(free)',IULA_cal['frequency'][6]/IULA_cal['frequency'][1]], 
    ['defect free', context.count('defect free'),'P(free | defect)','C(defect free) / C(free)',IULA_cal['frequency'][7]/IULA_cal['frequency'][1]]
]
IULA_cal = pd.DataFrame(
    mock_sentence,
    columns=['word','frequency','P(Wk|Wk-1)','C(Wk-1|Wk)/C(Wk-1)',"evaluate"]
)

IULA_cal.tail(4)

,word,frequency,P(Wk|Wk-1),C(Wk-1|Wk)/C(Wk-1),evaluate
4,a defeat,0,P(defeat | a),C(a defeat)/C(defeat),0.0
5,a defect,21,P(defect | a),C(a defect)/C(defect),0.106061
6,defeat free,0,P(free | defeat),C(defeat free) / C(free),0.0
7,defect free,0,P(free | defect),C(defect free) / C(free),0.0


In [65]:
print("P(a defeat free) = ",IULA_cal["evaluate"][4]*IULA_cal["evaluate"][6])

P(a defeat free) =  0.0


In [66]:
print("P(a defeat free) = ",IULA_cal["evaluate"][5]*IULA_cal["evaluate"][7])

P(a defeat free) =  0.0


---

Ended Assignment